# Zillow Clustering Porject

## Executive Summary

Goals: Build a new regression model to predict logerror (log(Zestimate) - log(SalePrice)), and use clustering to see if we can find groups of similar observations to generate new features to be fed into our supervised model.

## Objectives for this project include:

Identify the drivers for logerrror by using clustering methodologies.

Document the process and analysis throughout the data science pipeline.

Demonstrate the information that was discovered.

In [1]:
#imports
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFE
import statsmodels.api as sm
from sklearn.metrics import explained_variance_score

import wrangle as w
import explore as e
import model as m

### Wrangling

Acquire Zillow data from Codeup database, and turn it into a .csv for quicker use

In [2]:
#get our data, turn into .csv, and check shape
df = w.get_zillow_data()
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,NaN,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,NaN,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,NaN,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,NaN,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,1970746,NaN,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [3]:
#Check how many missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77575 entries, 0 to 77574
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        222 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77575 non-null  float64
 3   heatingorsystemtypeid         49570 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      206 non-null    float64
 6   airconditioningtypeid         25006 non-null  float64
 7   parcelid                      77575 non-null  int64  
 8   id                            77575 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77575 non-null  float64
 11  bedroomcnt                    77575 non-null  float64
 12  buildingqualitytypeid         49809 non-null  float64
 13  c

Takeaways:

Lots of missing values Some columns have inaccurate content? Let's just clean it up for now, and we'll decide what we want to explore on later

Prepare our data for mostly-ready use. We're going to rename columns, make dummies, create new features, drop outliers, and drop columns and rows with more than 50% missing values.

In [4]:
#Clean zillow and then look at the df once again
df = w.prepare_zillow(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49727 entries, 0 to 77574
Data columns (total 47 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   propertylandusetypeid       49727 non-null  float64
 1   parcel_id                   49727 non-null  int64  
 2   id                          49727 non-null  int64  
 3   baths                       49727 non-null  float64
 4   beds                        49727 non-null  float64
 5   calculatedbathnbr           49713 non-null  float64
 6   sqft                        49727 non-null  float64
 7   finishedsquarefeet12        49546 non-null  float64
 8   fullbathcnt                 49713 non-null  float64
 9   latitude                    49727 non-null  float64
 10  longitude                   49727 non-null  float64
 11  lotsizesquarefeet           49727 non-null  float64
 12  propertycountylandusecode   49727 non-null  object 
 13  rawcensustractandblock      497

In [5]:
#split into train, validate, test. Our target is logerror
train, validate, test = w.train_validate_test_split(df, 'logerror', seed=1234)

In [6]:
#imputation preserves all cases by replacing missing data with an estimated value 
#based on other available information
#return our train, validate, and test sets with calculatedbathnbr, fullbathcnt, regionidcity, regionidzip,censustractandblock
#imputed with most frequent values
train, validate, test = w.impute_nulls(train, validate, test, strategy='most_frequent', 
                        col_list=['calculatedbathnbr', 'fullbathcnt', 'regionidcity', 'regionidzip', 'censustractandblock'])

In [7]:
#return our train, validate, and test sets with finishedsquarefeet12, lotsizesquarefeet, structuretaxvaluedollarcnt, 
#tax_value, land tax value dollar count, and tax amount imputed with the column's median value. 
train, validate, test = w.impute_nulls(train, validate, test, strategy='median', 
                        col_list=['finishedsquarefeet12', 'lotsizesquarefeet', 'structuretaxvaluedollarcnt', 
                                  'tax_value', 'landtaxvaluedollarcnt', 'tax_amount'])

In [8]:
#checking there are no more nulls in train valdiate and test
train.isna().sum().sum(), validate.isna().sum().sum(), test.isna().sum().sum()

(67, 47, 25)

In [9]:
#drop rows with remaining nulls from all of our split sets
train = train.dropna()
validate = validate.dropna()
test = test.dropna()

In [10]:
#assuring there are no more nulls and we can almost get to work
train.isna().sum().sum(), validate.isna().sum().sum(), test.isna().sum().sum()

(0, 0, 0)

In [11]:
#total rows and columns that are left to work with 
#train should have most dataset
#47 columns in each dataset
train.shape, validate.shape, test.shape

((27805, 47), (11908, 47), (9932, 47))

In [12]:
#split new train, validate, and test into X_train, y_train, X_validate, y_validate, X_test, y_test with logerror as our target
X_train, y_train, X_validate, y_validate, X_test, y_test = w.split_X_y(train, validate, test, 'logerror')

In [14]:
#get object columns, and numeric columns so we can scale our data
obj_cols = w.get_object_cols(X_train)
num_cols = w.get_numeric_X_cols(X_train, obj_cols)

In [15]:
#scale our X_train, X_validate, and X_test with standard scaler
X_train_scaled, X_validate_scaled, X_test_scaled = w.standard_scale_data(X_train[num_cols],X_validate[num_cols],X_test[num_cols])

Takeaways:

- Our data is now clean, and split into train, validate and test.
- We dropped any rows and columns with more than 50% missing values 
- Dropped, filled, or imputed for remaining missing values 
- We went from 77,575 rows to 49,727rows. At the start, we had 68 columns, after the data is cleaned, now 47 columns.

### Explore